In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

ton_iot = pd.read_csv('NF-ToN-IoT-v2.csv')


In [2]:
ton_iot.shape

(16940496, 45)

### Deletes features from 

In [3]:
categorical_feature = ['L4_SRC_PORT','L4_DST_PORT','IPV4_SRC_ADDR',
                       'IPV4_DST_ADDR','PROTOCOL',
                       'L7_PROTO','DNS_QUERY_TYPE','ICMP_TYPE'
                        ,'ICMP_IPV4_TYPE','DNS_TTL_ANSWER']


not_used = ['DNS_QUERY_ID','FTP_COMMAND_RET_CODE']

tagets = ['Attack','Label']

to_drop_corr_col = ['SERVER_TCP_FLAGS', 'MAX_TTL', 'MAX_IP_PKT_LEN', 'RETRANSMITTED_OUT_PKTS']

to_drop_VIF = ['NUM_PKTS_1024_TO_1514_BYTES', 'CLIENT_TCP_FLAGS']

#to_drop = categorical_feature+not_used+tagets + to_drop_corr_col + to_drop_VIF

#X_Numerical = ton_iot.drop(columns=to_drop)

to_drop = not_used+tagets + to_drop_corr_col + to_drop_VIF

X = ton_iot.drop(columns=to_drop)


In [4]:
X['SRC_TO_DST_SECOND_BYTES'] = X['SRC_TO_DST_SECOND_BYTES'].astype(int)
X['DST_TO_SRC_SECOND_BYTES'] = X['DST_TO_SRC_SECOND_BYTES'].astype(int)

y_multi =  ton_iot['Attack']

y_multi = y_multi.replace('Benign',0)
y_multi = y_multi.replace('scanning',1)
y_multi = y_multi.replace('xss',2)
y_multi = y_multi.replace('ddos',3)
y_multi = y_multi.replace('password',4)
y_multi = y_multi.replace('dos',5)
y_multi = y_multi.replace('injection',6)
y_multi = y_multi.replace('backdoor',7)
y_multi = y_multi.replace('mitm',8)
y_multi = y_multi.replace('ransomware',9)

y_multi.value_counts()


0    6099469
1    3781419
2    2455020
3    2026234
4    1153323
5     712609
6     684465
7      16809
8       7723
9       3425
Name: Attack, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

X_train, X, y_train, y_multi = train_test_split(X, y_multi, test_size=0.3, random_state=42)

In [6]:
del ton_iot
del X_train
del y_train
X = X.reset_index()
y_multi = y_multi.reset_index()

X = X.drop(columns='index')
y_multi = y_multi.drop(columns='index')

In [7]:
X_Numerical = X.drop(columns=categorical_feature)

## Scaling of Numerical value using  quantile

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_Numerical_sc = pd.DataFrame()

scaler.fit(X_Numerical)

X_Numerical_sc[X_Numerical.columns] = scaler.transform(X_Numerical)

In [9]:
X_Numerical_sc.shape

(5082149, 25)

In [10]:
print('Numerical')
print(25*0.8)
print(25*0.5)
print(25*0.2)
print('categorical')
print(10*0.8)
print(10*0.5)
print(10*0.2)

Numerical
20.0
12.5
5.0
categorical
8.0
5.0
2.0


In [11]:

print(y_multi.value_counts())

Attack
0         1829223
1         1134420
2          736976
3          607187
4          346493
5          213722
6          205682
7            5050
8            2369
9            1027
dtype: int64


## Scale data

In [16]:
sb

SelectKBest(k=5)

In [22]:
from sklearn.feature_selection import f_classif, SelectKBest

sb = SelectKBest(f_classif, k=5)
sb.fit(X_Numerical_sc,y_multi)
cols_idxs = sb.get_support(indices=True)
col_anova_20 = X_Numerical_sc.iloc[:,cols_idxs]

##########################
sb = SelectKBest(f_classif, k=13)
sb.fit(X_Numerical_sc,y_multi)
cols_idxs = sb.get_support(indices=True)
col_anova_50 = X_Numerical_sc.iloc[:,cols_idxs]

##########################
sb = SelectKBest(f_classif, k=20)
sb.fit(X_Numerical_sc,y_multi)
cols_idxs = sb.get_support(indices=True)
col_anova_80 = X_Numerical_sc.iloc[:,cols_idxs]

In [23]:
X_A_20 = col_anova_20
X_A_50 = col_anova_50
X_A_80 = col_anova_80

In [25]:
X_A_80.head(4)

,OUT_BYTES,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,MIN_TTL,LONGEST_FLOW_PKT,SHORTEST_FLOW_PKT,MIN_IP_PKT_LEN,SRC_TO_DST_SECOND_BYTES,DST_TO_SRC_SECOND_BYTES,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_BYTES,SRC_TO_DST_AVG_THROUGHPUT,DST_TO_SRC_AVG_THROUGHPUT,NUM_PKTS_UP_TO_128_BYTES,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_256_TO_512_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT
0,0.021877,1.258326,2.102289,1.014774,1.115044,1.178755,2.541508,-0.504485,0.302037,0.005282,-0.007178,-0.015589,-0.014225,-0.222571,-0.151353,-0.023640,-0.014986,0.105674,1.369679,3.059472
1,-0.009822,0.549419,2.102392,-0.215606,0.142225,1.178755,-0.434902,0.114000,0.763047,0.005264,-0.007273,-0.015589,-0.014225,-0.138412,-0.154822,-0.027083,-0.014986,-0.028758,1.369679,1.030844
2,-0.011495,-0.957010,-0.475672,-0.215606,-0.216182,-0.699691,-0.459706,-0.092162,-1.234663,0.005262,-0.007278,-0.015589,-0.014225,-0.187377,-0.155255,-0.028023,-0.014986,-0.028758,-0.483495,-0.575418
3,0.020353,1.258326,-0.475672,-0.215606,-0.216182,1.178755,2.541508,0.114000,0.763047,0.005394,-0.007183,-0.015589,-0.014225,2.917314,0.769108,-0.025205,-0.014986,0.038458,1.369679,0.913647


## Kendall

In [26]:
from scipy.stats import kendalltau
features = [col for col in X_Numerical_sc.columns if col ]
kendall_tau_scores = {}
for feature in features:
    kendall_tau, _ = kendalltau(X_Numerical_sc[feature], y_multi)
    kendall_tau_scores[feature] = kendall_tau
    
sorted_kendall_tau_scores = sorted(kendall_tau_scores.items(), key=lambda x: abs(x[1]), reverse=True)

col_kendall_20 = [feature for feature, score in sorted_kendall_tau_scores[:5]]
col_kendall_50 = [feature for feature, score in sorted_kendall_tau_scores[:13]]
col_kendall_80 = [feature for feature, score in sorted_kendall_tau_scores[:20]]


In [27]:
X_K_20 = X_Numerical_sc[col_kendall_20]
X_K_50 = X_Numerical_sc[col_kendall_50]
X_K_80 = X_Numerical_sc[col_kendall_80]
X_K_80.columns

Index(['SRC_TO_DST_SECOND_BYTES', 'IN_BYTES', 'LONGEST_FLOW_PKT', 'OUT_BYTES',
       'DST_TO_SRC_SECOND_BYTES', 'NUM_PKTS_UP_TO_128_BYTES', 'MIN_TTL',
       'OUT_PKTS', 'TCP_WIN_MAX_OUT', 'IN_PKTS', 'MIN_IP_PKT_LEN',
       'DST_TO_SRC_AVG_THROUGHPUT', 'TCP_WIN_MAX_IN', 'SHORTEST_FLOW_PKT',
       'DURATION_OUT', 'SRC_TO_DST_AVG_THROUGHPUT',
       'NUM_PKTS_512_TO_1024_BYTES', 'DURATION_IN',
       'NUM_PKTS_256_TO_512_BYTES', 'FLOW_DURATION_MILLISECONDS'],
      dtype='object')

In [28]:
X_cat = X[categorical_feature]

In [29]:
X_Numerical_sc.shape

(5082149, 25)

In [30]:
X_cat.shape

(5082149, 10)

# Optimize the features

### L4_SRC_PORT & L4_DST_PORT

Ports with numbers 0–1023 are called system or well-known ports; ports with numbers 1024-49151 are called user or registered ports, and ports with numbers 49152-65535 are called dynamic, private or ephemeral ports.[2] Both system and user ports are used by transport protocols (TCP, UDP, DCCP, SCTP) to identify an application or service.

In [31]:
well_known_port = list(range(1024))
registered_ports = list(range(1024,49152))
ephemeral_ports = list(range(49152,65535))

In [32]:
X_cat['L4_SRC_PORT_C'] = 'Others'
X_cat['L4_SRC_PORT_C'][X_cat['L4_SRC_PORT'].isin(well_known_port)] = 'well_known_port'
X_cat['L4_SRC_PORT_C'][X_cat['L4_SRC_PORT'].isin(registered_ports)] = 'registered_ports'
X_cat['L4_SRC_PORT_C'][X_cat['L4_SRC_PORT'].isin(ephemeral_ports)] = 'ephemeral_ports'

X_cat.drop(columns=['L4_SRC_PORT'],inplace=True)

X_cat['L4_SRC_PORT_C'].value_counts()

print(X_cat.shape)

(5082149, 10)


In [33]:
X_cat['L4_DST_PORT_C'] = 'Others'
X_cat['L4_DST_PORT_C'][X_cat['L4_DST_PORT'].isin(well_known_port)] = 'well_known_port'
X_cat['L4_DST_PORT_C'][X_cat['L4_DST_PORT'].isin(registered_ports)] = 'registered_ports'
X_cat['L4_DST_PORT_C'][X_cat['L4_DST_PORT'].isin(ephemeral_ports)] = 'ephemeral_ports'

X_cat.drop(columns=['L4_DST_PORT'],inplace=True)
X_cat['L4_DST_PORT_C'].value_counts()


well_known_port     2770564
registered_ports    1961342
ephemeral_ports      350224
Others                   19
Name: L4_DST_PORT_C, dtype: int64

#### Removerd features are :

to_drop_zero = ['RETRANSMITTED_IN_BYTES','RETRANSMITTED_IN_PKTS','RETRANSMITTED_OUT_BYTES','FTP_COMMAND_RET_CODE','ICMP_TYPE','DNS_TTL_ANSWER']
&&

['SERVER_TCP_FLAGS', 'MAX_TTL', 'MAX_IP_PKT_LEN', 'RETRANSMITTED_OUT_PKTS', 'ICMP_IPV4_TYPE']

## DNS

In [34]:
X_cat['DNS_QUERY_TYPE'] = X_cat['DNS_QUERY_TYPE'].astype('O')
list_proto = [0,1,2,12]
X_cat['DNS_QUERY_TYPE'][~X_cat['DNS_QUERY_TYPE'].isin(list_proto)] = 'Other'
X_cat['DNS_QUERY_TYPE'].value_counts()

0        4362734
1         470684
Other     223073
12         24231
2           1427
Name: DNS_QUERY_TYPE, dtype: int64

### PROTOCOL

In [35]:
X_cat['PROTOCOL'] = X_cat['PROTOCOL'].astype(int)

X_cat['PROTOCOL'].value_counts()

6     4328168
17     747388
1        5381
2         851
58        360
44          1
Name: PROTOCOL, dtype: int64

In [36]:
print(X_cat.shape)

(5082149, 10)


In [37]:
X_cat['PROTOCOL'].value_counts()

6     4328168
17     747388
1        5381
2         851
58        360
44          1
Name: PROTOCOL, dtype: int64

### L7_PROTO

In [38]:
X_cat['L7_PROTO'] = X_cat['L7_PROTO'].astype('O')
list_proto = [0,7,91,7.178,1,92,131,5.178]
X_cat['L7_PROTO'][~X_cat['L7_PROTO'].isin(list_proto)] = 'Other'
X_cat['L7_PROTO'].value_counts()

0.0      3087248
7.0      1200629
91.0      519746
Other     146243
7.178      59755
1.0        39477
92.0       11421
131.0       8978
5.178       8652
Name: L7_PROTO, dtype: int64

#### Adresses

### Adress Range

In [39]:
# Private adresses
# Class A: 10.0.0.0 to 10.255.255.255
# Class B: 172.16.0.0 to 172.31.255.255
# Class C: 192.168.0.0 to 192.168.255.255
from ipaddress import ip_address


def findIPs(start, end):
    start = ip_address(start)
    end = ip_address(end)
    result = []
    while start <= end:
        result.append(str(start))
        start += 1
    return result
print('classe_a process ------------------------------->')
plage_classe_a=findIPs('10.0.0.0', '10.255.255.255')
print('classe_b process ------------------------------->')
plage_classe_b=findIPs('172.16.0.0', '172.31.255.255')
print('classe_c process ------------------------------->')
plage_classe_c=findIPs('192.168.0.0', '192.168.255.255')

classe_a process ------------------------------->
classe_b process ------------------------------->
classe_c process ------------------------------->


In [40]:
import numpy as np
X_cat['IPV4_DST_ADDR_CLASSE'] = 'None'
print('classe_c process ------------------------------->')
X_cat['IPV4_DST_ADDR_CLASSE'][X_cat['IPV4_DST_ADDR'].isin(plage_classe_c)] = 'classe_c'
print('classe_b process ------------------------------->')
X_cat['IPV4_DST_ADDR_CLASSE'][X_cat['IPV4_DST_ADDR'].isin(plage_classe_b)] = 'classe_b'
print('classe_a process ------------------------------->')
X_cat['IPV4_DST_ADDR_CLASSE'][X_cat['IPV4_DST_ADDR'].isin(plage_classe_a)] = 'classe_a'
print('localhost process ------------------------------->')
X_cat['IPV4_DST_ADDR_CLASSE'][X_cat['IPV4_DST_ADDR'] == '127.0.0.1'] = 'localhost'
print('public/other process ------------------------------->')
X_cat['IPV4_DST_ADDR_CLASSE'][X_cat['IPV4_DST_ADDR_CLASSE'] == 'None'] = 'public/other'
X_cat['IPV4_DST_ADDR_CLASSE'].value_counts()

classe_c process ------------------------------->
classe_b process ------------------------------->
classe_a process ------------------------------->
localhost process ------------------------------->
public/other process ------------------------------->


classe_c        4801805
public/other     278907
localhost          1429
classe_b              6
classe_a              2
Name: IPV4_DST_ADDR_CLASSE, dtype: int64

In [41]:
import numpy as np
X_cat['IPV4_SRC_ADDR_CLASSE'] = 'None'
print('classe_c process ------------------------------->')
X_cat['IPV4_SRC_ADDR_CLASSE'][X_cat['IPV4_SRC_ADDR'].isin(plage_classe_c)] = 'classe_c'
print('classe_b process ------------------------------->')
X_cat['IPV4_SRC_ADDR_CLASSE'][X_cat['IPV4_SRC_ADDR'].isin(plage_classe_b)] = 'classe_b'
print('classe_a process ------------------------------->')
X_cat['IPV4_SRC_ADDR_CLASSE'][X_cat['IPV4_SRC_ADDR'].isin(plage_classe_a)] = 'classe_a'
print('localhost process ------------------------------->')
X_cat['IPV4_SRC_ADDR_CLASSE'][X_cat['IPV4_SRC_ADDR'] == '127.0.0.1'] = 'localhost'
print('public/other process ------------------------------->')
X_cat['IPV4_SRC_ADDR_CLASSE'][X_cat['IPV4_SRC_ADDR_CLASSE'] == 'None'] = 'public/other'
X_cat['IPV4_SRC_ADDR_CLASSE'].value_counts()

classe_c process ------------------------------->
classe_b process ------------------------------->
classe_a process ------------------------------->
localhost process ------------------------------->
public/other process ------------------------------->


classe_c        5036473
classe_b          31158
public/other      13048
localhost          1431
classe_a             39
Name: IPV4_SRC_ADDR_CLASSE, dtype: int64

Remove IPV4_SRC_ADDR & IPV4_DST_ADDR and Keep IPV4_SRC_ADDR IPV4_DST_ADDR

In [42]:
X_cat.drop(columns=['IPV4_SRC_ADDR','IPV4_DST_ADDR'], inplace=True)

## Optimze the 3 features ICMP_TYPE, ICMP_IPV4_TYPE, DNS_TTL_ANSWER

In [43]:
import math
infini = math.inf

X_cat['ICMP_TYPE'] = X_cat['ICMP_TYPE'].astype('O')
X_cat['ICMP_IPV4_TYPE'] = X_cat['ICMP_IPV4_TYPE'].astype('O')
X_cat['DNS_TTL_ANSWER'] = X_cat['DNS_TTL_ANSWER'].astype('O')

X_cat['ICMP_TYPE'][~(X_cat['ICMP_TYPE']==0)]=1
X_cat['ICMP_IPV4_TYPE'][~(X_cat['ICMP_IPV4_TYPE']==0)]=1
X_cat['DNS_TTL_ANSWER'][~(X_cat['DNS_TTL_ANSWER']==0)]=1

print(f"ICMPTYPE {X_cat['ICMP_TYPE'].value_counts()}, \n ICMP_IPV4_TYPE  {X_cat['ICMP_IPV4_TYPE'].value_counts()}, \n  DNS_TTL_ANSWER  {X_cat['DNS_TTL_ANSWER'].value_counts()}")



ICMPTYPE 0    4997371
1      84778
Name: ICMP_TYPE, dtype: int64, 
 ICMP_IPV4_TYPE  0    4997371
1      84778
Name: ICMP_IPV4_TYPE, dtype: int64, 
  DNS_TTL_ANSWER  0    4749724
1     332425
Name: DNS_TTL_ANSWER, dtype: int64


In [44]:
X_cat.head(4)

,PROTOCOL,L7_PROTO,DNS_QUERY_TYPE,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_TTL_ANSWER,L4_SRC_PORT_C,L4_DST_PORT_C,IPV4_DST_ADDR_CLASSE,IPV4_SRC_ADDR_CLASSE
0,6,7.0,0,0,0,0,ephemeral_ports,well_known_port,classe_c,classe_c
1,6,91.0,0,0,0,0,registered_ports,well_known_port,classe_c,classe_c
2,6,0.0,0,0,0,0,registered_ports,registered_ports,classe_c,classe_c
3,6,7.178,0,0,0,0,ephemeral_ports,well_known_port,public/other,classe_c


# Feature Selection Filter-based

In [45]:
from sklearn.feature_selection import chi2

# Convert string values in categorical features to numerical
X_cat = X_cat.replace('Other',999)
X_cat = X_cat.replace('Others',999)
X_cat = X_cat.replace('well_known_port',0)
X_cat = X_cat.replace('registered_ports',1)
X_cat = X_cat.replace('ephemeral_ports',2)



X_cat = X_cat.replace('localhost',0)
X_cat = X_cat.replace('classe_a',1)
X_cat = X_cat.replace('classe_b',2)
X_cat = X_cat.replace('classe_c',3)
X_cat = X_cat.replace('public/other',4)


X_cat = X_cat.astype(int)



#X.dropna(inplace=True)

#X = X.astype(int)

chi2_corr = chi2(X_cat,y_multi)


In [46]:
list(X_cat.columns[np.argsort(-chi2_corr[0])])

['DNS_QUERY_TYPE',
 'L7_PROTO',
 'PROTOCOL',
 'L4_DST_PORT_C',
 'DNS_TTL_ANSWER',
 'ICMP_TYPE',
 'ICMP_IPV4_TYPE',
 'IPV4_DST_ADDR_CLASSE',
 'L4_SRC_PORT_C',
 'IPV4_SRC_ADDR_CLASSE']

In [47]:
columns_ranked_chi2 = list(X_cat.columns[np.argsort(-chi2_corr[0])])

## Ranking of features using CHI2

# We will select 3 pourcentage from the rest of the features
categorical
- 8.0
- 5.0
- 2.0

### Generate the 3 databases for chi 2

In [48]:
X_chi2_20 = X_cat[columns_ranked_chi2[:2]]
X_chi2_50 =  X_cat[columns_ranked_chi2[:5]]
X_chi2_80 =  X_cat[columns_ranked_chi2[:8]]

dummy_feature = ['L4_SRC_PORT_C','L4_DST_PORT_C','IPV4_SRC_ADDR_CLASSE',
                 'IPV4_DST_ADDR_CLASSE','PROTOCOL','L7_PROTO']
list_dummy_20 = list(set( list(X_chi2_20.columns)).intersection(dummy_feature))
list_dummy_50 = list(set( list(X_chi2_50.columns)).intersection(dummy_feature))
list_dummy_80 = list(set( list(X_chi2_80.columns)).intersection(dummy_feature))


X_chi2_20 = pd.get_dummies(X_chi2_20, columns=list_dummy_20)
X_chi2_50 = pd.get_dummies(X_chi2_50, columns=list_dummy_50)
X_chi2_80 = pd.get_dummies(X_chi2_80, columns=list_dummy_80)

In [49]:
X_chi2_80.shape

(5082149, 27)

# MUTUAL INFORMATION

In [50]:
from sklearn.feature_selection import mutual_info_classif

mi_class = mutual_info_classif(X_cat, y_multi)


In [51]:
list(X_cat.columns[np.argsort(-mi_class)])

['L4_DST_PORT_C',
 'L7_PROTO',
 'PROTOCOL',
 'IPV4_DST_ADDR_CLASSE',
 'IPV4_SRC_ADDR_CLASSE',
 'DNS_QUERY_TYPE',
 'L4_SRC_PORT_C',
 'DNS_TTL_ANSWER',
 'ICMP_IPV4_TYPE',
 'ICMP_TYPE']

In [52]:
columns_ranked_mi = list(X_cat.columns[np.argsort(-mi_class)])

In [53]:
columns_ranked_mi

['L4_DST_PORT_C',
 'L7_PROTO',
 'PROTOCOL',
 'IPV4_DST_ADDR_CLASSE',
 'IPV4_SRC_ADDR_CLASSE',
 'DNS_QUERY_TYPE',
 'L4_SRC_PORT_C',
 'DNS_TTL_ANSWER',
 'ICMP_IPV4_TYPE',
 'ICMP_TYPE']

In [54]:
X_MI_20 = X_cat[columns_ranked_mi[:2]]
X_MI_50 =  X_cat[columns_ranked_mi[:5]]
X_MI_80 =  X_cat[columns_ranked_mi[:8]]

list_dummy_20 = list(set( list(X_MI_20.columns)).intersection(dummy_feature))
list_dummy_50 = list(set( list(X_MI_50.columns)).intersection(dummy_feature))
list_dummy_80 = list(set( list(X_MI_80.columns)).intersection(dummy_feature))


X_MI_20 = pd.get_dummies(X_MI_20, columns=list_dummy_20)
X_MI_50 = pd.get_dummies(X_MI_50, columns=list_dummy_50)
X_MI_80 = pd.get_dummies(X_MI_80, columns=list_dummy_80)

In [55]:
y_multi.shape


(5082149, 1)

## mRMR

In [56]:
from mrmr import mrmr_classif
selected_features_20 = mrmr_classif(X=X_cat, y=y_multi, K=2)
selected_features_50 = mrmr_classif(X=X_cat, y=y_multi, K=5)
selected_features_80 = mrmr_classif(X=X_cat, y=y_multi, K=8)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:05<00:00,  1.49it/s]


In [57]:
X_mrmr_80 = X_cat[selected_features_80]
X_mrmr_50 = X_cat[selected_features_50]
X_mrmr_20 = X_cat[selected_features_20]

list_dummy_20 = list(set( list(X_mrmr_20.columns)).intersection(dummy_feature))
list_dummy_50 = list(set( list(X_mrmr_50.columns)).intersection(dummy_feature))
list_dummy_80 = list(set( list(X_mrmr_80.columns)).intersection(dummy_feature))

X_mrmr_20 = pd.get_dummies(X_mrmr_20, columns=list_dummy_20)
X_mrmr_50 = pd.get_dummies(X_mrmr_50, columns=list_dummy_50)
X_mrmr_80 = pd.get_dummies(X_mrmr_80, columns=list_dummy_80)

In [58]:
selected_features_80

['PROTOCOL',
 'IPV4_DST_ADDR_CLASSE',
 'ICMP_TYPE',
 'DNS_QUERY_TYPE',
 'L4_DST_PORT_C',
 'L7_PROTO',
 'DNS_TTL_ANSWER',
 'ICMP_IPV4_TYPE']

### Decision Tree

### MI

In [59]:
X_A_20.shape

(5082149, 5)

In [60]:
X_A_chi2_20 = pd.concat([X_A_20,X_chi2_20],axis=1)
X_A_chi2_20.shape

(5082149, 14)

In [61]:
X_A_chi2_20 = pd.concat([X_A_20,X_chi2_20],axis=1)
X_A_chi2_50 = pd.concat([X_A_50,X_chi2_50],axis=1)
X_A_chi2_80 = pd.concat([X_A_80,X_chi2_80],axis=1)

X_A_MI_20 = pd.concat([X_A_20,X_MI_20],axis=1)
X_A_MI_50= pd.concat([X_A_50,X_MI_50],axis=1)
X_A_MI_80= pd.concat([X_A_80,X_MI_80],axis=1)


X_K_chi2_20 = pd.concat([X_K_20,X_chi2_20],axis=1)
X_K_chi2_50 = pd.concat([X_K_50,X_chi2_50],axis=1)
X_K_chi2_80 = pd.concat([X_K_80,X_chi2_80],axis=1)

X_K_MI_20 = pd.concat([X_K_20,X_MI_20],axis=1)
X_K_MI_50 = pd.concat([X_K_50,X_MI_50],axis=1)
X_K_MI_80 = pd.concat([X_K_80,X_MI_80],axis=1)





In [62]:
X_K_mRMR_20 = pd.concat([X_K_20,X_mrmr_20],axis=1)
X_K_mRMR_50 = pd.concat([X_K_50,X_mrmr_50],axis=1)
X_K_mRMR_80 = pd.concat([X_K_80,X_mrmr_80],axis=1)


X_A_mRMR_20 = pd.concat([X_A_20,X_mrmr_20],axis=1)
X_A_mRMR_50 = pd.concat([X_A_50,X_mrmr_50],axis=1)
X_A_mRMR_80 = pd.concat([X_A_80,X_mrmr_80],axis=1)

In [63]:
print(X_A_chi2_20.shape)
print(X_A_chi2_50.shape)
print(X_A_chi2_80.shape)
print(X_A_MI_20.shape)
print(X_A_MI_50.shape)
print(X_A_MI_80.shape)
print(X_K_chi2_20.shape)
print(X_K_chi2_50.shape)
print(X_K_chi2_80.shape)
print(X_K_MI_20.shape)
print(X_K_MI_50.shape)
print(X_K_MI_80.shape)


(5082149, 14)
(5082149, 33)
(5082149, 47)
(5082149, 17)
(5082149, 41)
(5082149, 54)
(5082149, 14)
(5082149, 33)
(5082149, 47)
(5082149, 17)
(5082149, 41)
(5082149, 54)


In [64]:

print(X_K_mRMR_20.shape)
print(X_K_mRMR_50.shape)
print(X_K_mRMR_80.shape)


print(X_A_mRMR_20.shape)
print(X_A_mRMR_50.shape)
print(X_A_mRMR_80.shape)


(5082149, 16)
(5082149, 30)
(5082149, 47)
(5082149, 16)
(5082149, 30)
(5082149, 47)


In [58]:
import sys

# Get a snapshot of the current local variables
local_vars_snapshot = locals().copy()

# Sort local variables by size
sorted_local_vars = sorted(local_vars_snapshot.items(), key=lambda x: sys.getsizeof(x[1]), reverse=True)

print("Local Variables (sorted by size):")
for var_name, var_value in sorted_local_vars:
    print(f"{var_name}: {sys.getsizeof(var_value)} bytes")

# Get the current global variables
global_vars = globals()

# Sort global variables by size
sorted_global_vars = sorted(global_vars.items(), key=lambda x: sys.getsizeof(x[1]), reverse=True)

print("\nGlobal Variables (sorted by size):")
for var_name, var_value in sorted_global_vars:
    print(f"{var_name}: {sys.getsizeof(var_value)} bytes")


Local Variables (sorted by size):
X: 2005908332 bytes
X_Numerical_sc: 1016429964 bytes
X_A_MI_80: 1016429964 bytes
X_K_MI_80: 1016429964 bytes
X_A_chi2_80: 1011347815 bytes
X_K_chi2_80: 1011347815 bytes
X_Numerical: 975772772 bytes
X_A_80: 813144004 bytes
X_K_80: 813144004 bytes
X_A_chi2_50: 660679534 bytes
X_K_chi2_50: 660679534 bytes
X_A_MI_50: 650515236 bytes
X_K_MI_50: 650515236 bytes
X_A_50: 528543660 bytes
X_K_50: 528543660 bytes
X_A_chi2_20: 264271912 bytes
X_K_chi2_20: 264271912 bytes
X_A_MI_20: 259189763 bytes
X_K_MI_20: 259189763 bytes
X_A_20: 203286124 bytes
X_K_20: 203286124 bytes
X_cat: 203286124 bytes
X_MI_80: 203286124 bytes
X_chi2_80: 198203975 bytes
plage_classe_a: 142713400 bytes
X_chi2_50: 132136038 bytes
X_MI_50: 121971740 bytes
X_chi2_20: 60985952 bytes
X_MI_20: 55903803 bytes
y_multi: 40657356 bytes
plage_classe_b: 8448728 bytes
plage_classe_c: 562488 bytes
registered_ports: 385080 bytes
ephemeral_ports: 131128 bytes
well_known_port: 8248 bytes
_17: 2063 bytes
_34

In [65]:
del plage_classe_c
del registered_ports
del ephemeral_ports
del well_known_port

In [66]:

dummy_feature = ['L4_SRC_PORT_C','L4_DST_PORT_C','IPV4_SRC_ADDR_CLASSE',
                 'IPV4_DST_ADDR_CLASSE','PROTOCOL','L7_PROTO']
X_cat_dm = pd.get_dummies(X_cat, columns=dummy_feature)

In [67]:


X =  pd.concat([X_Numerical_sc,X_cat_dm],axis=1)
X.shape

(5082149, 61)

In [68]:
datasets = {"X_A_chi2_20":X_A_chi2_20,
"X_A_chi2_50":X_A_chi2_50,
"X_A_chi2_80":X_A_chi2_80,
"X_A_MI_20":X_A_MI_20,
"X_A_MI_50":X_A_MI_50,
"X_A_MI_80":X_A_MI_80,

"X_K_chi2_20":X_K_chi2_20,
"X_K_chi2_50":X_K_chi2_50,
"X_K_chi2_80":X_K_chi2_80,
"X_K_MI_20":X_K_MI_20,
"X_K_MI_50":X_K_MI_50,
"X_K_MI_80":X_K_MI_80,
"X_A_mRMR_20":X_A_mRMR_20,
"X_A_mRMR_50":X_A_mRMR_50,
"X_A_mRMR_80":X_A_mRMR_80,
"X_K_mRMR_20":X_K_mRMR_20,
"X_K_mRMR_50":X_K_mRMR_50,
"X_K_mRMR_80":X_K_mRMR_80,
"X":X
}

In [ ]:
"X_A_mRMR_20":X_A_mRMR_20,
"X_A_mRMR_50":X_A_mRMR_50,
"X_A_mRMR_80":X_A_mRMR_80,
"X_K_mRMR_20":X_K_mRMR_20,
"X_K_mRMR_50":X_K_mRMR_50,
"X_K_mRMR_80":X_K_mRMR_80,
"X":X


In [63]:
del X_A_chi2_20,
del X_A_chi2_50,
del X_A_chi2_80,
del X_A_MI_20,
del X_A_MI_50,
del X_A_MI_80,

del X_K_chi2_20,
del X_K_chi2_50,
del X_K_chi2_80,
del X_K_MI_20,
del X_K_MI_50,
del X_K_MI_80,


In [ ]:
del X_A_mRMR_20,
del X_A_mRMR_50,
del X_A_mRMR_80,

del X_K_mRMR_20,
del X_K_mRMR_50,
del X_K_mRMR_80

In [64]:
datasets.keys()

dict_keys(['X_A_chi2_20', 'X_A_chi2_50', 'X_A_chi2_80', 'X_A_MI_20', 'X_A_MI_50', 'X_A_MI_80', 'X_K_chi2_20', 'X_K_chi2_50', 'X_K_chi2_80', 'X_K_MI_20', 'X_K_MI_50', 'X_K_MI_80'])

### Multiclassification

## Building the models 

In [65]:
from sklearn import tree
from sklearn.model_selection import KFold
import json
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer,cohen_kappa_score,matthews_corrcoef,accuracy_score,precision_score,recall_score


def my_grid_result_df(clf,hyperparams,data_name,algo_name,X,y):
    
    scoring = {"Accuracy": make_scorer(accuracy_score),
               "F1-Score":'f1_macro' ,
               "Recall":  make_scorer(recall_score,average='macro'),
              # "Precision": make_scorer(precision_score, average='macro'),
               "MCC":make_scorer(matthews_corrcoef),
               "Kappa":make_scorer(cohen_kappa_score)
              }
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
    search = GridSearchCV(clf, hyperparams, scoring=scoring, n_jobs=-1, cv=cv_inner, refit="Kappa")
    grid_result = search.fit(X, y)

    json_dump = json.dumps(grid_result.cv_results_,cls=NumpyEncoder)
    # Serializing json

    # Writing to sample.json
    with open(f"./result_multi/{algo_name}__{data_name}.json", "w") as outfile:
        outfile.write(json_dump)


def create_all_result(clf,algo_name,hyperparams,datasets,y):
    for k,v in datasets.items():
        now = datetime.datetime.now()
        print(f'======== process======= dataset : {k} \t time is {now}')
        my_grid_result_df(clf,hyperparams,k,algo_name,v,y)
        now = datetime.datetime.now()
        print(f'======== Done  ======== dataset : {k} \t time is {now}')
        print('\n')

In [66]:
y_multi.value_counts()

Attack
0         1829223
1         1134420
2          736976
3          607187
4          346493
5          213722
6          205682
7            5050
8            2369
9            1027
dtype: int64

In [68]:
del large_dataset

In [69]:
from joblib import dump, load

# Example large dataset

# Save the dataset to a file
file_path = 'large_dataset.joblib'
dump(datasets, file_path)



KeyboardInterrupt: 

In [70]:
del loaded_dataset

NameError: name 'loaded_dataset' is not defined

In [78]:
from sklearn import tree
from sklearn.model_selection import KFold
import json
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer,cohen_kappa_score,matthews_corrcoef,accuracy_score,precision_score,recall_score


def my_grid_result_df(clf,hyperparams,data_name,algo_name,X,y):
    
    scoring = {"Accuracy": make_scorer(accuracy_score),
               "F1-Score":'f1_macro' ,
               "Recall":  make_scorer(recall_score,average='macro'),
              # "Precision": make_scorer(precision_score, average='macro'),
               "MCC":make_scorer(matthews_corrcoef),
               "Kappa":make_scorer(cohen_kappa_score)
              }
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
    search = GridSearchCV(clf, hyperparams, scoring=scoring, cv=cv_inner, refit="Kappa")
    grid_result = search.fit(X, y)

    json_dump = json.dumps(grid_result.cv_results_,cls=NumpyEncoder)
    # Serializing json

    # Writing to sample.json
    with open(f"./result_multi/{algo_name}__{data_name}.json", "w") as outfile:
        outfile.write(json_dump)


def create_all_result(clf,algo_name,hyperparams,datasets,y):
    for k,v in datasets.items():
        now = datetime.datetime.now()
        print(f'======== process======= dataset : {k} \t time is {now}')
        my_grid_result_df(clf,hyperparams,k,algo_name,v,y)
        now = datetime.datetime.now()
        print(f'======== Done  ======== dataset : {k} \t time is {now}')
        print('\n')

In [81]:
import json
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


In [ ]:
import datetime
from sklearn.ensemble import RandomForestClassifier

hyperparams = {'n_estimators':[50, 100, 150,200]}





print('======================= RandomForestClassifier =====================')
clf = RandomForestClassifier(random_state=0)
create_all_result(clf,'RandomForestClassifier',hyperparams,datasets,y_multi)
print_skip()




======================= RandomForestClassifier =====================
======== process======= dataset : X_A_chi2_20 	 time is 2024-05-17 23:04:54.882771
======== Done  ======== dataset : X_A_chi2_20 	 time is 2024-05-18 02:14:00.227469


======== process======= dataset : X_A_chi2_50 	 time is 2024-05-18 02:14:00.237524
======== Done  ======== dataset : X_A_chi2_50 	 time is 2024-05-18 07:55:02.855916


======== process======= dataset : X_A_chi2_80 	 time is 2024-05-18 07:55:02.866615
